In [1]:
# Imports Required Libraries
import numpy as np
import os
import pandas as pd
import pickle
import statsmodels.api as sm

from fastapi import FastAPI
from fastapi import Request

from math import ceil

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [2]:
def load_models():
    '''
    Returns loaded in, pre-trained Prediction, SimpleImputer and StandardScalar models
    '''
    
    mdl = sm.load('../models/glm_final_model.pickle')
    si = pickle.load(open('../models/glm_simple_imputer.pickle', 'rb'))
    ss = pickle.load(open('../models/glm_standard_scalar.pickle', 'rb'))
    
    return mdl, si, ss

In [3]:
def transform_json_to_df(obj):
    '''
    Returns a DataFrame containing raw JSON data
    
    Keyword Arguments:
    obj -- Raw JSON data in the data type form of a String, List or Dictionary
    '''
    
    if isinstance(obj, str):
        if obj[0] == '[' and obj[len(obj) - 1] == ']':
            df = pd.read_json(obj, orient = 'records')
        elif obj[0] == '{' and obj[len(obj) - 1] == '}':
            obj = '[' + obj + ']'
            df = pd.read_json(obj, orient = 'records')
        else:
            df = pd.DataFrame()
    elif isinstance(obj, list):
        df = pd.DataFrame.from_dict(obj)
    elif isinstance(obj, dict):
        obj = [obj]
        df = pd.DataFrame.from_dict(obj)
    else:
        df = pd.DataFrame()
    
    return df

In [4]:
def batch_df(max_batch_size, df):
    '''
    Returns
    
    Keyword Arguments:
    max_batch_size -- 
    df -- 
    '''
    
    df_batches = []
    
    num_rows = len(df.index)
    
    if num_rows > max_batch_size:
        
        num_batches = ceil(num_rows / max_batch_size)
        
        for batch_num in range(num_batches): 
            starting_index = batch_num * max_batch_size
            #print(f'Starting Index: {starting_index}')
            
            if batch_num != num_batches - 1:
                ending_index = (batch_num + 1) * max_batch_size
                #print(f'Ending Index: {ending_index}')
                batch_df = df.iloc[starting_index:ending_index].copy(deep = True)
            
            else:
                batch_df = df.iloc[starting_index:].copy(deep = True)
                
            df_batches.append(batch_df)
    
    else:
        df_batches.append(df.copy(deep = True))
        
    return df_batches

In [5]:
def format_df_column_variables(df):
    '''
    Returns a DataFrame after having transformed those columns, whose values 
        consisted of those of the String data type and representing quantitative 
        data, into values being that of the Float data type after removing any
        non mathematically interpretable symbols 
    
    Keyword Arguments:
    df -- A DataFrame containing raw JSON data
    '''
    
    # Formats the 'x12' and 'x63' columns', consisting of the String data type, 
    #    and respectively representing first monetary then percentage values,
    #    into the Float data type so as to be able to later on apply mathematical
    #    work upon said columns' values
    
    columns_to_format = ['x12', 'x63']
    
    unwanted_symbols = ['$', ',', '(', ')', '%']
    
    for clmn in columns_to_format:
        for usymb in unwanted_symbols:
            if usymb != '(':
                df[clmn] = df[clmn].str.replace(usymb, '', regex = True)
            else:
                df[clmn] = df[clmn].str.replace(usymb, '-', regex = True)
        df[clmn] = df[clmn].astype(float)
    
    return df

In [6]:
def impute_missing_df_data(si, df):
    '''
    Returns a DataFrame with no column variables whose data is of a qualitative nature, 
        as well as having filled in any remaining blank, NaN, NULL or otherwise missing 
        values via the usage of an imported, pre-trained SimpleImputer using a mean based 
        strategy
        
    Keyword Arguments:
    si -- A pre-trained, imported SimpleImputer
    df -- A formatted DataFrame
    '''
    
    # Columns containing qualitative data that are dropped ahead of the imputation phase
    #    as the SimpleImputer cannot properly function upon them
    qual_clmns = ['x5', 'x31', 'x81', 'x82']

    df = pd.DataFrame(si.transform(df.drop(columns = qual_clmns)), 
                      columns = df.drop(columns = qual_clmns).columns)
    
    return df

In [7]:
def scale_df_data(ss, df):
    '''
    Returns a DataFrame whose column variable values have all been scaled via a 
        standardization method for the purpose of feature scaling, utilizing an
        imported, pre-trained StandardScalar
    
    Keyword Arguments:
    ss -- A pre-trained, imported StandardScalar
    df -- A formatted DataFrame without any blank, NaN, NULL or otherwise missing values
    '''
    
    # Of particular interest and focus is that of the 'x12' column representing
    #    monetary values which tend to outscale all other column variable values
    #    by some orders of magnitude
    
    df = pd.DataFrame(ss.transform(df), 
                      columns = df.columns)
    
    return df

In [8]:
def create_df_dummy_column_variables_new(df1, df2):
    '''
    Returns a DataFrame with dummy variables for those column variables consisting of
        qualitative data, as well as whose column variables of numeric quantitative data
        have no missing values and are scaled
        
    Keyword Arguments:
    df1 -- A DataFrame containing the original raw JSON data in order to retrieve those column
           variable values of a qualitative nature previously dropped and must now be 
           dummified
    df2 -- A DataFrame with no missing column variable values and whose said values have already been scaled
    '''
    
    # A list of column variable names of a quantitative nature which require dummification
    vars_to_dummify = ['x5', 'x31', 'x81', 'x82']
    dummy_dfs = [df2]
    
    for var in vars_to_dummify:

        var_dummy_vars_df = pd.get_dummies(df1[var], 
                                        drop_first = True, 
                                        prefix = var, 
                                        prefix_sep = '_', 
                                        dummy_na = True)
        
        dummy_dfs.append(var_dummy_vars_df)

    df2 = pd.concat(dummy_dfs, axis = 1, sort = False)

    return df2

In [9]:
def filter_df_column_variables(ordr_clmn_names_lst, df):
    '''
    Returns a DataFrame containing only those column variables required by the pre-trained model
        for predictions, filtering out from the given DataFrame only said columns
    
    Keyword Arguments:
    ordr_clmn_names_lst -- A list of DataFrame column variable names required by the pre-trained model
    df -- The DataFrame with dummy variables, and whose quantitative values have been scaled and 
          have none missing
    '''
    
    necessary_clmn_vars_set = set(ordr_clmn_names_lst)
    avlbl_clmn_vars_set = set(df.columns)

    # Depending upon the type and amount of JSON data originally having been passed in, 
    #    not all of the desired dummy variables will always be successfully generated, 
    #    necessitating their inclusion afterwards via the code below
    if necessary_clmn_vars_set.issubset(avlbl_clmn_vars_set) == False:
        nan_df = pd.DataFrame(np.nan, index = range(df.shape[0]), columns = ordr_clmn_names_lst)
        df = df.combine_first(nan_df)
        df = df.fillna(0)

    #df = df[ordr_clmn_names_lst].copy()
    df = df[ordr_clmn_names_lst].copy(deep = True)  
    
    return df

In [10]:
def extract_transform_input_data_pipeline(df, si, ss, ordr_clmn_names_lst):
    '''
    Returns a transformed DataFrame originally filled with raw JSON data, transforming said data via
        imputation, to fill in any and all missing values, scaling, and the creation of dummy
        variables for those qualitative column variables which require such action
    
    Keyword Arguments:
    df -- A DataFrame containing raw JSON data
    si -- A pre-trained, imported SimpleImputer
    ss -- A pre-trained, imported StandardScalar
    ordr_clmn_names_lst -- A list of DataFrame column variable names required by the pre-trained model
    '''
    
    if df.shape[0] < 1:
        filter_dummy_scaled_imputed_df = pd.DataFrame()
 
    else:
        
        # Resets the index to avoid concatenation issues with dummy variable DataFrames
        df.reset_index(drop = True, inplace = True)
        
        batch_df = df.copy(deep = True)
        
        formatted_df = format_df_column_variables(batch_df)

        imputed_df = impute_missing_df_data(si, formatted_df)

        scaled_imputed_df = scale_df_data(ss, imputed_df)


        dummy_scaled_imputed_df = create_df_dummy_column_variables_new(df, scaled_imputed_df)


        filter_dummy_scaled_imputed_df = filter_df_column_variables(ordr_clmn_names_lst, 
                                                                    dummy_scaled_imputed_df)
        
    return filter_dummy_scaled_imputed_df

In [11]:
def extract_transform_df_batches(df_batches, si, ss, ordr_clmn_names_lst):
    '''
    Returns
    
    Keyword Arguments:
    df_batches -- 
    si -- A pre-trained, imported SimpleImputer
    ss -- A pre-trained, imported StandardScalar
    ordr_clmn_names_lst -- A list of DataFrame column variable names required by the pre-trained model
    '''
    
    transformed_df_batches = []
    
    for batch in df_batches:
        transformed_batch = extract_transform_input_data_pipeline(batch, si, ss, ordr_clmn_names_lst)
        transformed_df_batches.append(transformed_batch)
        
    return transformed_df_batches

In [12]:
def merge_df_batches(df_batches):
    '''
    Returns
    
    Keyword Arguments:
    df_batches -- 
    '''
    
    df = pd.concat(df_batches, ignore_index = True)
        
    return df

In [13]:
def predict_outcome(df, mdl, alphanum_ord_clmn_var_names_lst):
    '''
    Returns a JSON message containing either the model's predicted outcomes, 
        marked as 'business_outcome', predicted probability, marked as 'p_hat',
        and the inputs in the alphanumerical order of their variables' names or,
        should the predicted probability be under that of 75%, a message indicating
        as such; if no valid JSON data as far as the application can tell is passed in, 
        then a JSON encoded error message will be returned instead
        
    Keyword Arguments:
    df -- A DataFrame containing only those 25 column variables required by the pre-trained model
    mdl -- A pre-trained, imported prediction model
    alphanum_ord_clmn_var_names_lst -- A list of DataFrame column variable names required by 
                                       the pre-trained model ordered alphanumerically
    '''
    
    num_rows_df = df.shape[0]
    
    if num_rows_df == 0:
        
        return {'message': 'ERROR - No valid JSON data available for prediction.'}
    
    else:
    
        output_msgs_lst = []
    
        for row in range(num_rows_df):
            predicted_outcome = 0
            mdl_inputs = {}

            predicted_probability = mdl.predict(df.iloc[row])[0]

            if predicted_probability >= 0.75:
                predicted_outcome = 1
                
                for var in alphanum_ord_clmn_var_names_lst:
                    mdl_inputs[var] = df.iloc[row][var]

                mdl_predictions = {'business_outcome': str(predicted_outcome), 
                                    'p_hat': str(predicted_probability)}

                prediction_msg = mdl_predictions | mdl_inputs

                output_msgs_lst.append(prediction_msg)

            else:
                #output_msgs_lst.append({'message': 'Business outcome probability too low.'})
                pass
        
        return output_msgs_lst

In [14]:
def main(rows_per_batch, input_json):
    '''
    Returns a single event or a list of events as a JSON message containing the business outcome, 
        probability of said outcome, along with the input variables which led to said outcome 
        in alphanumerical order for all those predictions which met the minimum standard of 75% 
        chance of a successful sale to a potential buying customer
    
    Keyword Arguments:
    input_json -- Raw JSON data
    '''
    json = input_json
    
    # List of the already properly ordered column variables required by the pre-trained
    #    model in order for it to carry out accurate predictions
    final_df_column_variable_names_order = [
        'x5_saturday', 'x81_July', 'x81_December', 'x31_japan', 'x81_October', 
        'x5_sunday', 'x31_asia', 'x81_February', 'x91', 'x81_May', 
        'x5_monday', 'x81_September', 'x81_March', 'x53', 'x81_November', 
        'x44', 'x81_June', 'x12', 'x5_tuesday', 'x81_August', 
        'x81_January', 'x62', 'x31_germany', 'x58', 'x56']
    
    alphanumerically_sorted_df_column_variable_names = sorted(final_df_column_variable_names_order)
    
    mdl, si, ss = load_models()
    
    df = transform_json_to_df(json)
    
    df_batches = batch_df(rows_per_batch, df)
    
    #batch_details(df_batches)
    
    transformed_df_batches = extract_transform_df_batches(df_batches, 
                                                          si, 
                                                          ss, 
                                                          final_df_column_variable_names_order)
    
    print(f'Number Of Bacthes: {len(transformed_df_batches)}')
    
    batch_details(transformed_df_batches)
        
    transformed_df = merge_df_batches(transformed_df_batches)
    print(f'Merge Batches Total Rows: {len(transformed_df.index)}')
    
    json_output_message = predict_outcome(transformed_df, 
                                          mdl, 
                                          alphanumerically_sorted_df_column_variable_names)
        
    return json_output_message

In [15]:
# For Debugging & Testing Purposes
raw_testing_data1 = pd.read_csv(os.path.join('../testing', 'exercise_26_test.csv'))

raw_testing_data2 = pd.read_csv(os.path.join('../testing', 'exercise_26_test.csv'), nrows = 1)
raw_testing_data3 = pd.read_csv(os.path.join('../testing', 'exercise_26_test.csv'), nrows = 10)
raw_testing_data4 = pd.read_csv(os.path.join('../testing', 'exercise_26_test.csv'), nrows = 100)
raw_testing_data5 = pd.read_csv(os.path.join('../testing', 'exercise_26_test.csv'), nrows = 1000)

#raw_testing_data1.head()
#raw_testing_data2.head()
#raw_testing_data3.head()
#raw_testing_data4.head()
#raw_testing_data5.head()

In [16]:
sample_raw_json_10000_rows = raw_testing_data1.to_json(orient = 'records')

sample_raw_json_1_row_v2 = raw_testing_data2.to_json(orient = 'records')
sample_raw_json_10_rows = raw_testing_data3.to_json(orient = 'records')
sample_raw_json_100_rows = raw_testing_data4.to_json(orient = 'records')
sample_raw_json_1000_rows = raw_testing_data5.to_json(orient = 'records')

In [17]:
sample_raw_json_1_row_v1 = "{\"x0\":0.042317,\"x1\":-3.344721,\"x2\":4.6351242122,\"x3\":-0.5983959993,\"x4\":-0.6477715046,\"x5\":\"monday\",\"x6\":0.184902,\"x7\":46.690015,\"x8\":3.034132,\"x9\":0.364704,\"x10\":14.260733,\"x11\":-1.559332,\"x12\":\"$5,547.78\",\"x13\":0.520324,\"x14\":31.212255,\"x15\":4.891671,\"x16\":0.357763,\"x17\":14.766366,\"x18\":-17.467243,\"x19\":0.224628,\"x20\":0.096752,\"x21\":1.305564,\"x22\":0.353632,\"x23\":3.909028,\"x24\":-91.273052,\"x25\":1.396952,\"x26\":4.401593,\"x27\":0.443086,\"x28\":14.048787,\"x29\":-0.932243,\"x30\":5.255472,\"x31\":\"germany\",\"x32\":0.54199153,\"x33\":2.98948039,\"x34\":-1.78334189,\"x35\":0.80127315,\"x36\":-2.60231221,\"x37\":3.39682926,\"x38\":-1.22322646,\"x39\":-2.20977636,\"x40\":-68.69,\"x41\":522.25,\"x42\":-428.69,\"x43\":381.37,\"x44\":0.0197503,\"x45\":0.75116479,\"x46\":0.8630479008,\"x47\":-1.0383166613,\"x48\":-0.2726187635,\"x49\":-0.3430207259,\"x50\":0.3109008666,\"x51\":-0.797841974,\"x52\":-2.0390175153,\"x53\":0.87182889,\"x54\":0.14373012,\"x55\":-1.15212514,\"x56\":-2.1703139704,\"x57\":-0.267842962,\"x58\":0.212110633,\"x59\":1.6926559407,\"x60\":-0.9522767913,\"x61\":-0.8625864974,\"x62\":0.0748487158,\"x63\":\"36.29%\",\"x64\":3.47125327,\"x65\":-3.16656509,\"x66\":0.65446814,\"x67\":14.60067029,\"x68\":-20.57521013,\"x69\":0.71083785,\"x70\":0.16983767,\"x71\":0.55082127,\"x72\":0.62814576,\"x73\":3.38608078,\"x74\":-112.45263714,\"x75\":1.48370808,\"x76\":1.77035368,\"x77\":0.75702363,\"x78\":14.75731742,\"x79\":-0.62550355,\"x80\":null,\"x81\":\"October\",\"x82\":\"Female\",\"x83\":-0.7116680715,\"x84\":-0.2653559892,\"x85\":0.5175495907,\"x86\":-1.0881027092,\"x87\":-1.8188638198,\"x88\":-1.3584469527,\"x89\":-0.654995195,\"x90\":-0.4933042262,\"x91\":0.373853,\"x92\":0.94143481,\"x93\":3.54679834,\"x94\":-99.8574882,\"x95\":0.403926,\"x96\":1.65378726,\"x97\":0.00771459,\"x98\":-32.02164582,\"x99\":-60.3127828}"

In [18]:
# Saves raw JSON data files for proper testing when Docker becomes involved
#with open(os.path.join('../testing', 'sample_raw_json_1_row_v1.json'), 'w') as file:
#    file.write(sample_raw_json_1_row_v1)

In [19]:
# Retrieves slices of raw testing data of various sizes whilst 
#    converting and saving them into a JSON format
#sample_raw_json_10000_rows = raw_testing_data1.to_json(os.path.join('../testing', 'sample_raw_json_10000_rows.json'), orient = 'records')

#sample_raw_json_1_row_v2 = raw_testing_data2.to_json(os.path.join('../testing', 'sample_raw_json_1_row_v2.json'), orient = 'records')
#sample_raw_json_10_rows = raw_testing_data3.to_json(os.path.join('../testing', 'sample_raw_json_10_rows.json'), orient = 'records')
#sample_raw_json_100_rows = raw_testing_data4.to_json(os.path.join('../testing', 'sample_raw_json_100_rows.json'), orient = 'records')
#sample_raw_json_1000_rows = raw_testing_data5.to_json(os.path.join('../testing', 'sample_raw_json_1000_rows.json'), orient = 'records')

In [20]:
raw_testing_data3.iloc[3:6]

x0        x1        x2        x3        x4        x5        x6  \
3 -0.065676  1.892277  4.818741  0.640313  1.944562    friday  0.208718   
4 -0.357126 -1.852161  5.367849 -0.069869 -0.641455  saturday  0.940286   
5 -1.142132  0.012474  3.917673 -0.722627  0.412905    friday  0.975457   

          x7        x8        x9  ...       x90       x91       x92       x93  \
3  73.573314  4.929132  0.116004  ...  0.305243 -0.099213  0.712234  3.853489   
4  72.773335       NaN  0.191044  ...  0.617258  0.307445  0.376738  3.306958   
5  86.782049  2.480363  0.229048  ...  1.023861  0.507509  0.485926  3.852836   

         x94       x95       x96       x97        x98         x99  
3 -91.650053  0.499861  2.804358  0.627921 -32.190043  103.192597  
4 -99.557140  1.275527  1.476482  0.122798 -32.957087 -111.509168  
5 -94.374115  1.193276  0.700849  0.885308 -32.111276   61.841748  

[3 rows x 100 columns]

In [21]:
def batch_details(df_batches):
    row_cnt = 0
    
    for index in range(len(df_batches)):
        
        print(f'Rows in Batch #{index}: {len((df_batches[index]).index)}')
        
        row_cnt += len((df_batches[index]).index)
    
    print(f'Total Rows: {row_cnt}')

In [22]:
def results_details(json_res, print_res):
    print(f'Results Type: {type(json_res)}\n')
    print(f'Results Length: {len(json_res)}\n')
    
    if isinstance(json_res, list) and print_res == True:
        for result in json_res:
            print(f'{result}\n')

In [23]:
# Results Length: Expected = 0
json_results1 = main(500, sample_raw_json_1_row_v1)
results_details(json_results1, True)

Number Of Bacthes: 1
Rows in Batch #0: 1
Total Rows: 1
Merge Batches Total Rows: 1
Results Type: <class 'list'>

Results Length: 0



In [24]:
# Results Length: Expected = 0
json_results2 = main(500, sample_raw_json_1_row_v2)
results_details(json_results2, True)

Number Of Bacthes: 1
Rows in Batch #0: 1
Total Rows: 1
Merge Batches Total Rows: 1
Results Type: <class 'list'>

Results Length: 0



In [25]:
# Results Length: Expected = 2
json_results3 = main(500, sample_raw_json_10_rows)
results_details(json_results3, True)

Number Of Bacthes: 1
Rows in Batch #0: 10
Total Rows: 10
Merge Batches Total Rows: 10
Results Type: <class 'list'>

Results Length: 2

{'business_outcome': '1', 'p_hat': '0.8227526744741207', 'x12': -0.9529553121992512, 'x31_asia': 0.0, 'x31_germany': 0.0, 'x31_japan': 0.0, 'x44': -0.018906143399875407, 'x53': -1.5974242627501094, 'x56': 1.6483959900371876, 'x58': 0.599511719639024, 'x5_monday': 0.0, 'x5_saturday': 0.0, 'x5_sunday': 0.0, 'x5_tuesday': 1.0, 'x62': 0.7274434380391581, 'x81_August': 0.0, 'x81_December': 0.0, 'x81_February': 0.0, 'x81_January': 0.0, 'x81_July': 0.0, 'x81_June': 0.0, 'x81_March': 0.0, 'x81_May': 0.0, 'x81_November': 1.0, 'x81_October': 0.0, 'x81_September': 0.0, 'x91': -0.3633580994460479}

{'business_outcome': '1', 'p_hat': '0.7539305190612656', 'x12': 1.1180442609776802, 'x31_asia': 0.0, 'x31_germany': 1.0, 'x31_japan': 0.0, 'x44': 1.0491037054747567, 'x53': -1.0452443300416803, 'x56': -0.9224261424370558, 'x58': 0.41258044127697047, 'x5_monday': 0.0, 'x5

In [26]:
# Results Length: Expected = 24
json_results4 = main(500, sample_raw_json_100_rows)
results_details(json_results4, False)

Number Of Bacthes: 1
Rows in Batch #0: 100
Total Rows: 100
Merge Batches Total Rows: 100
Results Type: <class 'list'>

Results Length: 24



In [27]:
# Results Length: Expected = 215
json_results5 = main(500, sample_raw_json_1000_rows)
results_details(json_results5, False)

Number Of Bacthes: 2
Rows in Batch #0: 500
Rows in Batch #1: 500
Total Rows: 1000
Merge Batches Total Rows: 1000
Results Type: <class 'list'>

Results Length: 215



In [28]:
# Results Length: Expected = 2013
json_results6 = main(500, sample_raw_json_10000_rows)
results_details(json_results6, False)

Number Of Bacthes: 20
Rows in Batch #0: 500
Rows in Batch #1: 500
Rows in Batch #2: 500
Rows in Batch #3: 500
Rows in Batch #4: 500
Rows in Batch #5: 500
Rows in Batch #6: 500
Rows in Batch #7: 500
Rows in Batch #8: 500
Rows in Batch #9: 500
Rows in Batch #10: 500
Rows in Batch #11: 500
Rows in Batch #12: 500
Rows in Batch #13: 500
Rows in Batch #14: 500
Rows in Batch #15: 500
Rows in Batch #16: 500
Rows in Batch #17: 500
Rows in Batch #18: 500
Rows in Batch #19: 500
Total Rows: 10000
Merge Batches Total Rows: 10000
Results Type: <class 'list'>

Results Length: 2013

